In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0023636817932128906
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 12.669281243807848
200000 18.429878673342674
300000 19.374801927007375
400000 16.86018045211632
500000 19.895545208389127
600000 20.876649994099306
700000 17.489980415223346
800000 16.979653031046777
900000 10.326077866211422
1000000 15.833257771498667
1100000 17.487979635654014
1200000 17.553284631020638
1300000 16.88309719369271
1400000 18.001323414536166
1500000 17.73516935969885
1600000 18.0916944489165
1700000 17.619813690710547
1800000 9.031285160212118
1900000 17.186276414050784
2000000 19.555620225400112
2100000 19.60612660961753
2200000 20.17927550100016
2300000 18.404500447648413
2400000 17.8369610434109
2500000 18.51316882523918
2600000 17.339844447899072
2700000 8.002187743073096
2800000 19.926409649266514
2900000 17.77317655548599
3000000 19

29400000 14.286579839997172
29500000 15.688588698872913
29600000 15.872112128678792
29700000 16.07729896106282
29800000 17.256058146381918
29900000 9.220238921280794
30000000 15.281684465168864
30100000 16.368623188425264
30200000 16.336249261250813
30300000 16.333774177339507
30400000 8.543187443900221
30500000 16.008492341224763
30600000 15.48835538701846
30700000 16.256583004694996
30800000 15.325792551250252
30900000 14.36688974722427
31000000 10.697042791383607
31100000 10.60853189597511
31200000 15.603052307865015
31300000 14.65180945564244
31400000 15.324729335351565
31500000 15.681227634877493
31600000 16.251124805975042
31700000 9.023585768415177
31800000 15.912624901005625
31900000 16.207889101419234
32000000 16.281981375123927
32100000 17.452555362492035
32200000 7.6759394242229
32300000 17.593890206575516
32400000 16.07467853766196
32500000 16.88590048363471
32600000 15.13260010522309
32700000 14.80584334230718
32800000 7.11534129148627
32900000 13.7415823239907
33000000 14

59100000 12.002262178083342
59200000 12.07054440305653
59300000 12.876997388824366
59400000 5.463361211720367
59500000 11.774525491492296
59600000 12.233025169389327
59700000 11.702438015957146
59800000 6.68164418129172
59900000 12.716858407562562
60000000 12.149816813638866
60100000 13.019676521407016
60200000 13.027856771306567
60300000 13.272789159123372
60400000 13.867674153177678
60500000 13.230841159148643
60600000 12.550881511476904
60700000 13.079391523123094
60800000 5.653668959816026
60900000 11.52756244900189
61000000 12.072519581771727
61100000 10.849629314601367
61200000 6.868319251040588
61300000 12.302863125498247
61400000 13.061923110036425
61500000 9.036001131794274
61600000 9.62808724779293
61700000 12.931371843534082
61800000 12.709997215593496
61900000 12.719254926630189
62000000 13.590957134228281
62100000 13.89877879988126
62200000 13.871458552662745
62300000 13.23927782445543
62400000 13.142427062023042
62500000 11.594959316398587
62600000 7.108145843992368
62700

88900000 10.262148080146366
89000000 10.307460950744582
89100000 10.082568712956856
89200000 10.774062755513645
89300000 8.692919651511202
89400000 6.19229198102197
89500000 4.096914200549802
89600000 9.049898509570395
89700000 6.86407020349438
89800000 10.293430623318008
89900000 10.208218428044297
90000000 9.373850803625873
90100000 10.27671710316273
90200000 10.608242126419064
90300000 10.517460227520942
90400000 11.25079025099636
90500000 11.921038650558804
90600000 9.72233995124051
90700000 9.755878411339468
90800000 10.689672235525931
90900000 9.724911217634391
91000000 9.712107795252322
91100000 8.00875500306772
91200000 8.193014858165801
91300000 4.0814162801498375
91400000 5.7409804408802625
91500000 9.385321773749203
91600000 9.76168569953704
91700000 9.896426983222822
91800000 10.134780411786915
91900000 9.625777312794307
92000000 10.06634431002512
92100000 11.661270500539809
92200000 10.691480557228576
92300000 10.455425772467231
92400000 10.368115500630706
92500000 9.39866

118200000 8.77564585878831
118300000 7.985732894047511
118400000 8.040458629062398
118500000 8.069720177700978
118600000 8.162222870862035
118700000 9.098281175045514
118800000 9.115169592405428
118900000 8.508380101963537
119000000 8.050483901526622
119100000 8.131808702796043
119200000 8.16687393216892
119300000 8.140469080354523
119400000 8.570775231478214
119500000 5.078041289934106
119600000 3.856286421874189
119700000 4.161653577387343
119800000 7.0016157779432096
119900000 7.8467368323511675
120000000 8.073150907733678
120100000 7.860773487502624
120200000 7.662067926723083
120300000 8.809873202117712
120400000 8.748136265630063
120500000 9.050200325533131
120600000 8.836877116376886
120700000 8.469558369223112
120800000 8.599696091936297
120900000 8.493262479537874
121000000 8.17020685458169
121100000 8.554417237353348
121200000 8.290119900061974
121300000 4.720969394222614
121400000 3.436592947048207
121500000 4.588881237573793
121600000 7.662640474228693
121700000 8.600781436

147500000 6.043515859172332
147600000 4.307911994633455
147700000 5.518622158666078
147800000 5.883387776138378
147900000 5.86969730791086
148000000 3.1747862163712544
148100000 5.754959786540505
148200000 6.2746304594622355
148300000 3.097250464455515
148400000 5.987530375852396
148500000 6.147362443170725
148600000 6.6988352636228665
148700000 6.904236774471061
148800000 6.997034634925247
148900000 6.678802848119497
149000000 6.542006716090096
149100000 6.612671960324765
149200000 6.147823277616878
149300000 6.0016429394791855
149400000 3.3797244023907087
149500000 6.187309872514602
149600000 5.824983502775835
149700000 4.308606463995218
149800000 4.579506834585478
149900000 6.078309550843203
150000000 6.498631388851562
150100000 3.148319913020962
150200000 5.969651694300717
150300000 6.357506703501234
150400000 6.794813322517484
150500000 7.11137240496664
150600000 6.9864512246863555
150700000 6.666308277542498
150800000 6.607933973853678
150900000 6.394654873405367
151000000 6.2004

176700000 4.488779396410883
176800000 4.586043851482669
176900000 2.203326439933648
177000000 4.866171484767935
177100000 4.6097036999013525
177200000 4.472498732540038
177300000 4.782536966818521
177400000 4.289105925158312
177500000 2.5618236977565885
177600000 4.705468303960651
177700000 4.028523287476305
177800000 4.275185297018743
177900000 4.107076957795133
178000000 4.143854261337509
178100000 1.9009450718118848
178200000 4.073795194319209
178300000 4.08468714855001
178400000 4.509175757268724
178500000 4.525389825072438
178600000 4.263728759234163
178700000 2.43569938833042
178800000 4.60937701069212
178900000 4.7050363913193545
179000000 4.158368189986326
179100000 4.2631442106165585
179200000 3.4702047058722107
179300000 3.058846479545758
179400000 4.024555970748067
179500000 4.108880841919863
179600000 3.7313460047961007
179700000 3.7771504722080333
179800000 3.1617909744666877
179900000 2.458851477477627
180000000 3.73109275963668
180100000 4.225960115111702
180200000 3.830

205500000 1.5996154102364482
205600000 1.29674804150587
205700000 2.6167413005881803
205800000 2.5725331012486814
205900000 2.619394787537436
206000000 2.517289604968058
206100000 2.610616833280126
206200000 2.398980699472437
206300000 2.5237552594511077
206400000 1.4801178858477773
206500000 2.02820781122709
206600000 2.549693959088663
206700000 2.383875093627274
206800000 2.605904699549039
206900000 2.5551663536075058
207000000 2.423214807298382
207100000 2.370972511840585
207200000 1.8784214130322936
207300000 1.5027284894473552
207400000 1.4542729500005875
207500000 2.3354898895195126
207600000 2.299529648828427
207700000 2.384577984056516
207800000 2.2876777250534266
207900000 2.1750584450542427
208000000 2.2500095150946047
208100000 2.3264643275822334
208200000 0.9775285985326767
208300000 2.3167627238641253
208400000 2.3468590299689405
208500000 2.1558913444202443
208600000 2.176539377057622
208700000 2.3523264764040577
208800000 2.3164629758253197
208900000 2.2505659566403065
2

233500000 0.07852402667975757
233600000 0.12665758199448054
233700000 0.11247054238563776
233800000 0.12337191744539472
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.06882432820683387
234000000 0.02202992672628164
234100000 0.02281153342641062
234200000 0.0188413894077705
234300000 0.016754287683993577
234400000 0.016667785750256642
234500000 0.01690863594415784
234600000 0.012875595394223928
234700000 0.012139512955529822
234800000 0.01302611159432928
234900000 0.011042505871693293
235000000 0.010669968060079547
235100000 0.010274935504512655
235200000 0.010299947772949933
235300000 0.011599446803202232
235400000 0.009000679745806588
235500000 0.008615478732705116
235600000 0.008330244092235962
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.030904228494644168
235800000 0.04491096207221349
235900000 0.04970811229971714
236000000 0.0414236126451691
236100000 0.0348365330324173
236200000 0.009633769954482714
2363000